**Building Basic Model**

You can use this model for basic operation, you can try some more combinations to get more accurate output and faster operation the basic skeleton is given below

Including all the modules which we will be using

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Giving training and validation directory

In [ ]:
train_dir = ''
test_dir = ''

Using data gnerator for augmentation

In [ ]:
#Data Generator

#Performing some operations for more data from a single picture
train_datagen = ImageDataGenerator(
                                    rescale=1./255,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,             #if you want to divide test and validation then add parameter: validation_split=0.2, 8-2 split
                                    fill_mode='nearest')

#We can provide same eefects to test datagen but no need as we are only testing data
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                          train_dir,               # This is the source directory for training images
                          target_size=(150, 150),  # All images will be resized to 150x150
                          batch_size=20,           # Since we use binary_crossentropy loss, we need binary labels
                          class_mode='binary')     #For categorical use categorical in class mode
                          #subset='training')      #set as training data in case you have added validation split in the datagen

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
                          validation_dir,
                          target_size=(150, 150),
                          batch_size=20,
                          class_mode='binary')
                          #subset='validation')     #set as training data in case you have added validation split in the datagen



Making our actual model

In [ ]:
#Model Making

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),    #Change the shape as your requirement
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Dropout(0.2),                       #Helps to avoid overfitting of model by dropping some weights
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')      #Sigmoid is used data is in binary form for categorical use softmax
])


#Compile Model

model.compile(loss='binary_crossentropy',               #For categorical use categorical)crossentropy
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

Training our Data

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,  #steps = total_image_train / batch_size
      epochs=10,            #No of epochs
      validation_data=validation_generator,
      validation_steps=50)  #steps = total_image_test / batch_size

After traning it is good to save our model 

In [ ]:
#Save the model to use it later and to save time next time

model.save("MyBasicModel.h5")

For Use this model later

In [ ]:
import h5py
from tensorflow import keras

model = keras.models.load_model('/MyBasicModel.h5')     #Give path to your model

Now, we have prepared our model and we are now good to for predicting the image

In [ ]:
status = ['First','Second']   #Rename this as per you need eg. Cat or dog, house or hourse etc.

path = ''   #Path to image which we want to predict

img = image.load_img(path, target_size=(150, 150))

x = image.img_to_array(img)       #tensorflow require image in array
x = np.expand_dims(x, axis=0)     #dimmension of array should be 4 so we are expanding the dimmension o array

output = model.predict(x)
output = output[0][0]

tag = int(output)

print(status[tag])